In [ ]:
from imports import *
import traces_rdf
%matplotlib inline

In [ ]:
# Use traces with large trigger shifts -- P*S1 channels have -100 to -400 us shifts
trigShift = 'data/FastDMC/TriggerShift/iZIP5-TrigShift_00241024_0000.root'

In [ ]:
def bestI0(file, sensor, event, iChan):
    """Compute 'adaptive I0' for trace, scanning the supposed pre-trigger
       baseline for the range of values with the smallest RMS.  This
       should exclude the region where the real trace starts.  Returns
       computed I0 value, along with index of average and RMS value."""
    bins, trace = traces_rdf.loadTrace(file, event, iChan, sensor)

    start = 5                   # Need some bins to compute RMS

    rms = [np.std(trace[:i],ddof=1) for i in range(start,len(trace))]
    eom = [np.std(trace[:i])/sqrt(i) for i in range(start,len(trace))]
    ibest = eom.index(min(eom))+start

    return np.mean(trace[:ibest]), ibest, eom[ibest]

In [ ]:
event=0
iChan=7
adaptiveI0, ibase, err = bestI0(trigShift, "TES", event, iChan)

In [ ]:
print("Average from 0 to bin",ibase,"=", adaptiveI0,"+-",err)

In [ ]:
chans = traces_rdf.getChannels(trigShift, "TES", event)
for i in range(len(chans)):
    I0, ibase, err = bestI0(trigShift, "TES", event, i)
    print(chans[i],": I0 =",I0,"+-",err," to bin",ibase)

In [ ]:
bins, trace = traces_rdf.loadTrace(trigShift, event, iChan, "TES")
eom = [0]*5 + [np.std(trace[:i],ddof=1)/sqrt(i) for i in range(5,len(trace))]
runningI0 = [np.mean(trace[:i]) for i in range(1,len(trace))]

In [ ]:
plt.plot(bins, eom, label="EOM [0:time]")
plt.yscale('log')
plt.legend()

In [ ]:
plt.plot(bins, runningI0, label="Cumulative average")
plt.legend()